In [1]:
from keras.datasets import cifar100
import os
import sys
from six.moves import urllib
import gzip
import pickle
import numpy as np
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt



Using TensorFlow backend.


In [2]:
# size of MNIST
# print(x_train.shape)
# print(y_train.shape)
# print(x_test.shape)
# print(y_test.shape)

In [3]:
# # show data
# _, (ax1) = plt.subplots(1)
# sample_data = x_train[0]
# ax1.imshow(sample_data, cmap=plt.cm.Greys)

In [4]:

    def residual_block(self, input_layer, filter: List[int], channel: List[int], stride:int = 1) -> tf.Tensor:
        """
        input -> Conv -> BN -> ReLU -> Conv -> BN -> Addition -> ReLU -> output
        :param input_layer: Usually previous layer
        :param filter: (width<int>, height<int>) The size of the filter
        :param channel: [in_channels, out_channels]
        :param stride: The size of the s
        :return:
        """
        input_channel, output_channel = channel
        
        h = self.conv_bn(input_layer, filter = filter, channel=[input_channel, output_channel], stride = stride)
        h = tf.nn.relu(h)
        h = self.conv_bn(h, filter=filter, channel= [output_channel, output_channel], stride = stride)
        
        if input_channel != output_channel:
            # Input channel 과 output channel이 dimension이 다르기 때문에 projection 을 통해서 dimension을 맞춰준다.
            inp, _filter = self.conv(input_layer, filter=[1,1], channel=[input_channel,output_channel], stride = stride)
        else:
            inp = input_layer
        
        h = tf.add(h, inp)
        h = tf.nn.relu(h)
        self.layers.append(h)
        return h


In [5]:
def model(resnet: ResNet):
    with tf.variable_scope('input_scope'):
        h = init_block(filter=[7, 7], channel=[3, 64], max_pool=False)
        
    with tf.variable_scope('residual01'),tf.device('/device:GPU:0'):
        h = residual_block(h, filter=[3, 3], channel=[64, 64])
        h = resnet.residual_block(h, filter=[3, 3], channel=[64, 64])
        h = resnet.residual_block(h, filter=[3, 3], channel=[64, 64])
        h = resnet.residual_block(h, filter=[3, 3], channel=[64, 64])
        h = resnet.residual_block(h, filter=[3, 3], channel=[64, 64])
        h = resnet.residual_block(h, filter=[3, 3], channel=[64, 64])
        
    with tf.variable_scope('residual02'),tf.device('/device:GPU:0'):
        h = resnet.max_pool(h, kernel=[2, 2], stride=[2, 2])
        h = resnet.residual_block(h, filter=[3, 3], channel=[64, 128])
        h = resnet.residual_block(h, filter=[3, 3], channel=[128, 128])
        h = resnet.residual_block(h, filter=[3, 3], channel=[128, 128])
        h = resnet.residual_block(h, filter=[3, 3], channel=[128, 128])
        h = resnet.residual_block(h, filter=[3, 3], channel=[128, 128])
        h = resnet.residual_block(h, filter=[3, 3], channel=[128, 128])
        h = resnet.residual_block(h, filter=[3, 3], channel=[128, 128])
        h = resnet.residual_block(h, filter=[3, 3], channel=[128, 128])
        
    with tf.variable_scope('residual03'),tf.device('/device:GPU:0'):
        h = resnet.max_pool(h, kernel=[2, 2], stride=[2, 2])
        h = resnet.residual_block(h, filter=[3, 3], channel=[128, 256])
        h = resnet.residual_block(h, filter=[3, 3], channel=[256, 256])
        h = resnet.residual_block(h, filter=[3, 3], channel=[256, 256])
        h = resnet.residual_block(h, filter=[3, 3], channel=[256, 256])
        h = resnet.residual_block(h, filter=[3, 3], channel=[256, 256])
        h = resnet.residual_block(h, filter=[3, 3], channel=[256, 256])
        h = resnet.residual_block(h, filter=[3, 3], channel=[256, 256])
        h = resnet.residual_block(h, filter=[3, 3], channel=[256, 256])
        h = resnet.residual_block(h, filter=[3, 3], channel=[256, 256])
        h = resnet.residual_block(h, filter=[3, 3], channel=[256, 256])
        h = resnet.residual_block(h, filter=[3, 3], channel=[256, 256])
        h = resnet.residual_block(h, filter=[3, 3], channel=[256, 256])
    
    with tf.variable_scope('residual04'),tf.device('/device:GPU:0'):
        h = resnet.max_pool(h, kernel=[2, 2], stride=[2, 2])
        h = resnet.residual_block(h, filter=[3, 3], channel=[256, 512])
        h = resnet.residual_block(h, filter=[3, 3], channel=[512, 512])
        h = resnet.residual_block(h, filter=[3, 3], channel=[512, 512])
        h = resnet.residual_block(h, filter=[3, 3], channel=[512, 512])
        h = resnet.residual_block(h, filter=[3, 3], channel=[512, 512])
        h = resnet.residual_block(h, filter=[3, 3], channel=[512, 512])
    
    with tf.variable_scope('fc'),tf.device('/device:GPU:0'):
        h = resnet.avg_pool(h, kernel=[2, 2], stride=[2, 2])
        h = resnet.fc(h)
    return h

In [6]:
epoch_ = 10
batch = 100
save_interval = 5000
visualize_interval = 20

In [7]:
from keras.preprocessing.image import ImageDataGenerator
from queue import deque

def train(resnet, interval = visualize_interval):
    with tf.variable_scope('fc'):
        loss = resnet.loss()
        adam = tf.train.AdamOptimizer()
        train_op = adam.minimize(loss)
        resnet.compile()

    # Get Data
    (train_x, train_y), (test_x, test_y) = tf.keras.datasets.cifar100.load_data(label_mode='fine')

    # Image Augmentation
    datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, featurewise_center=True,
                                 featurewise_std_normalization=True)
    datagen.fit(train_x)

    iter_count = 0
    _losses = deque(maxlen=interval)
    for epoch in range(1, epoch_ + 1):
        sample_count = 0

        for i, (sample_x, sample_y) in enumerate(datagen.flow(train_x, train_y, batch_size=batch)):
            print(sample_x.shape, sample_y.shape)
            feed_dict = {resnet.x_ts: sample_x, resnet.y_ts: sample_y}
            _loss, _ = resnet.sess.run([loss, train_op], feed_dict=feed_dict)

            # Visualize
            _losses.append(_loss)
            iter_count += 1
            sample_count += 1
            if i % interval == 0:
                _loss = np.mean(_losses)
                print(f'[epoch:{epoch:02}] loss:{_loss:<7.4}  '
                      f'sample_count:{sample_count:<5}  '
                      f'iteration:{iter_count:<5}  ')

            # Save
            if iter_count % save_interval == 0:
                print(f'Model has been successfully saved at iteration = {iter_count}')
                resnet.save()


def evaluate(resnet, batch_size= batch):
    correct_prediction = tf.equal(tf.argmax(resnet.last_layer, 1), y=resnet.y_ts)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    resnet.compile()

    # Get Data
    (train_x, train_y), (test_x, test_y) = tf.keras.datasets.cifar100.load_data(label_mode='fine')

    accuracies = list()
    for i in range(0, 10000, batch_size):
        if i + batch_size < 10000:
            _acc = resnet.sess.run(accuracy, feed_dict={
                resnet.x_ts: test_x[i:i + batch_size],
                resnet.y_ts: test_y[i:i + batch_size]})
            accuracies.append(_acc)

    print('Accuracy', np.mean(accuracies))


def main():
    resnet = ResNet(batch= batch)
    model(resnet)

    print('Start Training')
    train(resnet)
#     if mode == 'train':
#         print('Start Training')
#         train(resnet)
#     elif mode == 'test':
#         print('Start Evaluation')
#         evaluate(resnet)


if __name__ == '__main__':
    main()

Start Training
(100, 32, 32, 3) (100, 1)


InvalidArgumentError: labels must be 1-D, but got shape [100,1]
	 [[node fc_1/cross_entropy/cross_entropy (defined at <ipython-input-4-0962b8d35352>:164)  = SparseSoftmaxCrossEntropyWithLogits[T=DT_FLOAT, Tlabels=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](fc/add, _arg_Placeholder_1_0_1/_783)]]
	 [[{{node fc_1/cross_entropy_mean/_789}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_14244_fc_1/cross_entropy_mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'fc_1/cross_entropy/cross_entropy', defined at:
  File "/home/pirl/.conda/envs/tf/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/pirl/.conda/envs/tf/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/pirl/.conda/envs/tf/lib/p

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()
tf.device('/device:GPU:0')